In [2]:
import numpy as np
import pandas as pd 
import glob
from tqdm import tqdm
import hashlib
import datetime

from rdflib import Namespace, Literal, URIRef
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

In [3]:
# today
today = datetime.date.today()
print(f"마지막으로 실행한 날짜는 {today} 입니다.")

마지막으로 실행한 날짜는 2022-08-26 입니다.


### (1) 건축물대장 표제부 데이터 불러오기

In [ ]:
%%time

columns = ["관리건축물대장PK","대장구분코드","대장구분코드명","대장종류코드","대장종류코드명","대지위치", \
            "도로명대지위치","건물명","시군구코드","법정동코드","대지구분코드","번","지","특수지명", \
            "블록","로트","외필지수","새주소도로코드","새주소법정동코드","새주소지상지하코드","새주소본번", \
            "새주소부번","동명","주부속구분코드","주부속구분코드명","대지면적(㎡)","건축면적(㎡)", \
            "건폐율(%)","연면적","용적률산정연면적","용적률","구조코드","구조코드명","기타구조", \
            "주용도코드","주용도코드명","기타용도","지붕코드","지붕코드명","기타지붕","세대수(세대)", \
            "가구수(가구)","높이(m)","지상층수","지하층수","승용승강기수","비상용승강기수","부속건축물수", \
            "부속건축물면적","총동연면적","옥내기계식대수(대)","옥내기계식면적","옥외기계식대수(대)", \
            "옥외기계식면적","옥내자주식대수(대)","옥내자주식면적","옥외자주식대수(대)","옥외자주식면적", \
            "허가일","착공일","사용승인일","허가번호년","허가번호기관코드","허가번호기관코드명", \
            "허가번호구분코드","허가번호구분코드명","호수(호)","에너지효율등급","에너지절감율", \
            "에너지EPI점수","친환경건축물등급","친환경건축물인증점수","지능형건축물등급","지능형건축물인증점수", \
            "생성일자","내진설계적용여부","내진능력"]

df = pd.read_csv("data/rawdata/building/표제부/mart_djy_03.txt", sep = "\|", engine='python', dtype=str, \
                    on_bad_lines='skip', keep_default_na=False, encoding = "cp949", header=None, names=columns)
df = df.replace({np.nan: None})

# preprocessing
df[df.columns] = df.apply(lambda x: x.str.strip())
print(f"총 {df.shape[0]} 개의 행수가 있습니다.")

In [ ]:
# 속성 rdf 파일을 만들기 위해 속성으로만 df 만들기

col = list(df.columns)
col_df = pd.DataFrame(col, columns=["속성명"])

### (2) RDF 변환

In [5]:
# namespace 
buid_def = "http://data.kisti.re.kr/administration/building/def/"
buid_id = "http://data.kisti.re.kr/administration/building/id/"
BUID = Namespace(buid_def)

# function (convert cell to rdf)
def cell(store, s, p, df_col, datatype = None, lang = None):
    if df_col != None:
        store.add((s, p, Literal(df_col, datatype=datatype, lang = lang)))
        
def uri(store, s, p, df_col, objClass = None, objURI = None) :
    if df_col != None :
        obj = URIRef(objURI + df_col) 
        store.add((s, p, obj))
        if objClass != None :
            store.add((obj, RDF.type, objClass))

### (3) 속성 만들기 

In [ ]:
%%time

# generate Graph()
g = Graph()
g.bind("buid", BUID)

#vocab 
for idx, row in tqdm(col_df.iterrows(), total=col_df.shape[0]):
    # base def 
    bui_uri = URIRef(buid_def + f"bp{idx+1}")
    g.add((bui_uri, RDFS.label, Literal(row["속성명"])))

# save as ttl
g.serialize(destination=f"final-data/ttl/building-property-{today}.ttl", format="turtle")

### (4) 건축물대장 표제부 변환하기

In [ ]:
%%time

# generate Graph()
g = Graph()
g.bind("buid", BUID)

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    # base id 
    uri_buid = URIRef(buid_def + row["관리건축물대장PK"])

    # type
    g.add((uri_buid, RDF.type, BUID.Building))

    # literal
    for p in range(len(df.columns)):
        cell(g, uri_buid, BUID[f"bp{p+1}"], row[f"{df.columns[p]}"])

# the number of triples
print(f"총 {len(g)} 개의 트리플이 있습니다.")

# save as ttl
g.serialize(destination=f"final-data/ttl/building-pyo-{today}.ttl", format="ttl")